In [6]:
library(limma)
library(edgeR)
library(gplots)

setwd("C:\\Users\\Derrick\\Documents\\DSC180B-PROJECT")
 

data<-read.delim("data/temp/all-.csv",header=TRUE)
# data<-read.delim("all-PreT1D-count.txt",header=TRUE)

# voom 
c<-data[,2:ncol(data)]
rownames(c)<-data[,1]
#colnames(c)<-c("M7_aTreg","M8_aTreg","M9_aTreg","M10_aTreg","M12_aTreg","M26_aTreg","M27_aTreg","M29_aTreg","M30_aTreg","P1_aTreg","P2_aTreg","P3_aTreg","P4_aTreg","P5_aTreg","P6_aTreg","P7_aTreg")
group<-factor(c(rep("Naive",7),rep("rTreg",7),rep("aTreg",7),rep("Ttm",7),rep("Tcm",7),rep("Tem",7)))

design2<-model.matrix(~0+group)
design1<-model.matrix(~group)
design<-cbind(design1[,1],design1[,3],design2[,1],design1[,4:6])
colnames(design)<-c("(Intercept)","grouprTreg","groupaTreg","groupTcm","groupTem","groupTtm")

d <- DGEList(counts=data,group=group)
#d <- DGEList(counts=c,group=group)
nf<-calcNormFactors(d$counts)
design<-model.matrix(~group)

nf<-calcNormFactors(d$counts)
y<-voom(d$counts,plot=TRUE,design,lib.size=colSums(d$counts)*nf)
fit <- lmFit(y,design)
fit <- eBayes(fit)
summary(decideTests(fit))
degene<-topTable(fit,coef=2,n=5)


# F test for Naive 2 rTreg
c1<-data[8:21]+0.5
group<-factor(c(rep("rTreg",7),rep("aTreg",7)))
#group<-factor(c(rep("Naive",7),rep("rTreg",7)))
d <- DGEList(counts=c1,group=group)
d$samples
dd <- d[rowSums(d$counts) >= 1.5*ncol(d), ]
d<-1e+06 * dd$count/expandAsMatrix(d$samples$lib.size,dim(dd))

d<-log2(d)

F<-data.frame(nrow(d))
x<-data.frame(nrow(d),2)
for (i in 1:nrow(d)) {
a<-7*abs(mean(d[i,1:7])-mean(d[i,1:14]))+7*abs(mean(d[i,8:14])-mean(d[i,1:14]))
bb = 0
        for (j in 1:7) {
        b<-abs(d[i,j]-mean(d[i,1:7]))
        bb = bb +b
        }

        for (j in 8:14) {
        b<-abs(d[i,j]-mean(d[i,8:14]))
        bb = bb +b
        }
	
F[i]<-12*a/bb

x[i,2]<-mean(d[i,1:7])
x[i,1]<-mean(d[i,8:14])

}

names(F)<-rownames(d)
rownames(x)<-rownames(d)
length(F[,F>9.6])

## 93 for rTreg 2 aTreg
# 109

mydist<-function(c) {
require(amap)
Dist(c,method="pearson")
 }
myclust<-function(c) { hclust(c,method='ward') }
heatmap.2(y,distfun=mydist,hclustfun=myclust,trace="none",cexRow=1.0,cexCol=1.2,las=2,col=greenred(30),lhei=c(1,7),scale="row",sepcolor="yellow", sepwidth = 0.1,margin=c(9,9))


# fold change
f<-x[names(-sort(-F))[1:93],1]-x[names(-sort(-F))[1:93],2]
names(f)<-names(-sort(-F))[1:93]
id<-c(names(f[f>=2]),names(f[-f>=2]))
y<-d[id,]
heatmap.2(y,distfun=mydist,hclustfun=myclust,trace="none",cexRow=1.0,cexCol=1.2,las=2,col=greenred(30),lhei=c(1,7),scale="row",sepcolor="yellow", sepwidth = 0.1,margin=c(9,9))
heatmap.2(x=y,distfun=mydist,dendrogram ="none",hclustfun=myclust,trace="none",density.info="none",cexRow=0.8,cexCol=1.2,las=2,col=greenred(30),lhei=c(1,6),sepcolor="yellow", sepwidth = 0.1,margin=c(7,10),scale="row",Colv = FALSE)
write.csv(y,file="Pre-T1D-aTreg2rTreg-intensity.csv")

de<-rownames(x)%in%id

plotMA(2*x,status=de,col=c("black", "red"),cex=0.8,legend=FALSE,main="Pre-T1D aTreg vs healthy aTreg")
legend("bottomright",legend=c("F<10","F>=10"),col=c("black","red"),pch=c(20,20))

f<-c(x[de,1]-x[de,2])
names(f)<-names(-sort(-F))[1:7]

fval<-as.numeric(F)
fold<-as.numeric(x[,1]-x[,2])
plot(fval,fold,main="F statistics vs fold change",xlab="F value",ylab="Fold change",pch=20,col="blue")
abline(h=c(2,-2),v=10,col="red")

# F test for aTrege 2 Naive
#c2<-cbind(data[1:7],data[15:21])+0.5
# F test for Ttm 2 Naive
# F test for Tcm 2 Naive
# F test for Tem 2 Naive
#c2<-cbind(data[1:7],data[22:28])+0.5
#c2<-cbind(data[1:7],data[29:35])+0.5
c2<-cbind(data[1:7],data[36:42])+0.5
#group<-factor(c(rep("Naive",7),rep("aTreg",7)))
group<-factor(c(rep("Naive",7),rep("Tcm",7)))
d <- DGEList(counts=c2,group=group)
d$samples
dd <- d[rowSums(d$counts) >= 1.5*14, ]
d<-1e+06 * dd$count/expandAsMatrix(d$samples$lib.size,dim(dd))

d<-log2(d)

F<-data.frame(nrow(d))
x<-data.frame(nrow(d),2)
for (i in 1:nrow(d)) {
a<-7*abs(mean(d[i,1:7])-mean(d[i,1:14]))+7*abs(mean(d[i,8:14])-mean(d[i,1:14]))
bb = 0
        for (j in 1:7) {
        b<-abs(d[i,j]-mean(d[i,1:7]))
        bb = bb +b
        }

        for (j in 8:14) {
        b<-abs(d[i,j]-mean(d[i,8:14]))
        bb = bb +b
        }

F[i]<-12*a/bb

x[i,2]<-mean(d[i,1:7])
x[i,1]<-mean(d[i,8:14])

}

names(F)<-rownames(d)
rownames(x)<-rownames(d)
length(F[,F>9.6])
## 93 for rTreg 2 aTreg
# 109 rTreg 2 Naive
# 117 aTreg 2 Naive
# 72 Ttm 2 Naive 
# 125 Tcm 2 Naive
# 197 for Tem 2 Naive
# fold change
# comm to Naive id<-c(id_aTreg,"hsa-miR-126*","hsa-mir-619","hsa-miR-30d*","hsa-mir-3929","hsa-mir-1273d")
f<-x[names(-sort(-F))[1:117],1]-x[names(-sort(-F))[1:117],2]
names(f)<-names(-sort(-F))[1:117]
id<-c(names(f[f>=2]),names(f[-f>=2]))
length(id)

# id<-c(id_PrTreg,id_PaTreg,id_PTtm,id_PTcm,id_PTem)
length(id)
# [1] 370
u_id<-unique(id)
# [1] 209
rownames(d)[rownames(d)%in%u_id]
#179 left
c<-data[,1:ncol(data)]+0.5
group<-factor(c(rep("Naive",7),rep("rTreg",7),rep("aTreg",7),rep("Ttm",7),rep("Tcm",7),rep("Tem",7)))
d <- DGEList(counts=c,group=group)
d$samples
dd <- d[rowSums(d$counts) >= 1.5*ncol(data), ]
d<-1e+06 * dd$count/expandAsMatrix(d$samples$lib.size,dim(dd))
d<-log2(d)

rownames(d)[rownames(d)%in%u_id]
X<-d[rownames(d)[rownames(d)%in%u_id],]
x.mean<-data.frame(179,6)
for ( i in 1:179 ){
 x.mean[i,1]<-mean(X[i,1:6])
 x.mean[i,2]<-mean(X[i,7:14])
 x.mean[i,3]<-mean(X[i,15:23])
 x.mean[i,4]<-mean(X[i,24:28])
 x.mean[i,5]<-mean(X[i,29:33])
 x.mean[i,6]<-mean(X[i,34:38])
 }

colnames(x.mean)<-c("Naive","rTreg","aTreg","Tcm","Tem","Ttm")
xx<- apply(x.mean,2,as.numeric)
rownames(xx)<-rownames(x.mean)
heatmap.2(xx,distfun=mydist,hclustfun=myclust,trace="none",cexRow=1.0,cexCol=1.0,las=2,col=greenred(30),lhei=c(1,6),scale="row",sepcolor="yellow", sepwidth = 0.1,margin=c(7,7))




ERROR: Error in `[.data.frame`(data, , 2:ncol(data)): undefined columns selected


In [7]:
# voom 
# c<-data[,2:ncol(data)]
# rownames(c)<-data[,1]
#colnames(c)<-c("M7_aTreg","M8_aTreg","M9_aTreg","M10_aTreg","M12_aTreg","M26_aTreg","M27_aTreg","M29_aTreg","M30_aTreg","P1_aTreg","P2_aTreg","P3_aTreg","P4_aTreg","P5_aTreg","P6_aTreg","P7_aTreg")
group<-factor(c(rep("Naive",7),rep("rTreg",7),rep("aTreg",7),rep("Ttm",7),rep("Tcm",7),rep("Tem",7)))

In [8]:
group

[1] Naive Naive Naive Naive Naive Naive Naive rTreg rTreg rTreg rTreg rTreg
[13] rTreg rTreg aTreg aTreg aTreg aTreg aTreg aTreg aTreg Ttm   Ttm   Ttm  
[25] Ttm   Ttm   Ttm   Ttm   Tcm   Tcm   Tcm   Tcm   Tcm   Tcm   Tcm   Tem  
[37] Tem   Tem   Tem   Tem   Tem   Tem  
Levels: aTreg Naive rTreg Tcm Tem Ttm

In [9]:
design2<-model.matrix(~0+group)
design1<-model.matrix(~group)
design<-cbind(design1[,1],design1[,3],design2[,1],design1[,4:6])
colnames(design)<-c("(Intercept)","grouprTreg","groupaTreg","groupTcm","groupTem","groupTtm")

In [12]:
design1

,(Intercept),groupNaive,grouprTreg,groupTcm,groupTem,groupTtm
1,1,1,0,0,0,0
2,1,1,0,0,0,0
3,1,1,0,0,0,0
4,1,1,0,0,0,0
5,1,1,0,0,0,0
6,1,1,0,0,0,0
7,1,1,0,0,0,0
8,1,0,1,0,0,0
9,1,0,1,0,0,0
10,1,0,1,0,0,0


In [11]:
design2

,groupaTreg,groupNaive,grouprTreg,groupTcm,groupTem,groupTtm
1,0,1,0,0,0,0
2,0,1,0,0,0,0
3,0,1,0,0,0,0
4,0,1,0,0,0,0
5,0,1,0,0,0,0
6,0,1,0,0,0,0
7,0,1,0,0,0,0
8,0,0,1,0,0,0
9,0,0,1,0,0,0
10,0,0,1,0,0,0


In [13]:
design

,(Intercept),grouprTreg,groupaTreg,groupTcm,groupTem,groupTtm
1,1,0,0,0,0,0
2,1,0,0,0,0,0
3,1,0,0,0,0,0
4,1,0,0,0,0,0
5,1,0,0,0,0,0
6,1,0,0,0,0,0
7,1,0,0,0,0,0
8,1,1,0,0,0,0
9,1,1,0,0,0,0
10,1,1,0,0,0,0


In [18]:
library(ggplot2)
library(reshape2)

In [26]:
all <-read.csv("data/temp/all.csv")

pre1 <- all[all$healthy == FALSE,]

countdata <- dcast(pre1, miRNA~sample, value.var="Total.miRNA.reads", fill=0)

# colnames(all)

ERROR: Error in dim(ordered) <- ns: dims [product 1] do not match the length of object [0]


In [27]:
all

X,miRNA,Chromosome,Position,Strand,Total.miRNA.reads,RPM..reads.per.million.,healthy,subset,sample
<int>,<fct>,<fct>,<fct>,<fct>,<int>,<int>,<fct>,<fct>,<fct>
0,hsa-miR-378c,chr10,132760851,-,7,1,True,Naive,GSM1088200_M7_Naive
1,hsa-mir-1321,chrX,85090785,+,26,6,True,Naive,GSM1088200_M7_Naive
2,hsa-miR-378d,chr8,94928250,-,2,1,True,Naive,GSM1088200_M7_Naive
3,hsa-miR-2110,chr10,115933864,-,5,1,True,Naive,GSM1088200_M7_Naive
4,hsa-mir-200b,chr1,1102484,+,2,1,True,Naive,GSM1088200_M7_Naive
5,hsa-miR-125a-5p,chr19,52196507,+,7,1,True,Naive,GSM1088200_M7_Naive
6,hsa-mir-671,chr7,150935507,+,1,1,True,Naive,GSM1088200_M7_Naive
7,hsa-miR-193a-5p,chr17,29887015,+,1,1,True,Naive,GSM1088200_M7_Naive
8,hsa-mir-378,chr5,149112388,+,4,1,True,Naive,GSM1088200_M7_Naive
